### Modelo de linguagem 

No tutorial de hoje, vamos explorar um pouco o conceito de modelo de linguagem na sua essência.
Durante a última semana, usamos a API da OpenAI pra consumir o GPT, que é um LLM, mas ainda não foi aprofundado o que é de fato um modelo de linguagem, e o que eles conseguem e não conseguem fazer.

#### O que é um modelo de linguagem?
Imagina que eu te dou a tarefa de completar a seguinte frase:

"A cor do céu é ___"

Você provavelmente vai saber que a palavra que falta é azul. Não necessariamente porque você sabe os fenômenos físicos por trás disso, mas porque algo na sua memória te diz que azul é a resposta.

Para treinar um modelo de linguagem utilizamos a mesma ideia. Nós forçamos o modelo a aprender a completar frases com os caracteres faltantes, até que em algum momento depois de ter ingerido muitos dados durante o treino, ele tenha a capacidade de gerar textos novos de acordo com o que ele aprendeu.

#### Como funciona por dentro?

Na prática, um modelo de linguagem trabalha com um vocabulário predefinido de tokens(que podem ser caracteres, palavras, ou partes de palavras). Cada token no vocabulário tem um índice único. Por exemplo, se nosso vocabulário tiver 200 tokens diferentes, o modelo precisa decidir qual desses 200 tokens deve vir em seguida em qualquer ponto do texto.

#### Previsão de probabilidades

Quando o modelo precisa prever o próximo token, ele gera um vetor de probabilidades com o mesmo tamanho do vocabulário(no nosso exemplo, 200 números). Cada número nesse vetor representa a probabilidade de cada token ser o próximo na sequência.
Essas probabilidades:
- são números entre 0 e 1
- são normalizadas usando uma função chamada softmax, garantindo que todas as probabilidades somem 1
- o token com a maior probabilidade é geralmente escolhido como a próxima previsão

Usando o primeiro exemplo que eu dei, temos a frase "A cor do céu é", o modelo poderia gerar probabilidades como:
- "azul": 0.85
- "vermelho': 0.05
- "verde": 0.03
- outros tokens de probabilidades menores 

Durante o aprendizado, o modelo ajusta essas probabilidades geradas para baterem com o que é esperado dos dados de treinamento. Cada vez que o modelo faz uma previsão errada, ele ajusta os parâmetros para aumentar a probabilidade do token correto e diminuir a dos errados.

#### Extra: Emergencia de Conhecimento 

Uma das áreas que mais existe estudo atualmente sobre LLMs é como eles desenvolvem conhecimentos que vão além da simples previsão de tokens. Embora treinemos estes modelos com o objetivo específico de prever o próximo token em uma sequência, eles acabam emergindo com capacidades e conhecimentos muito mais sofisticados.

Uma pesquisa recente da Anthropic trouxe algumas conclusões sobre como esse conhecimento se organiza dentro do modelo. Usando uma técnica chamada "sparse autoencoder", os pesquisadores conseguiram analisar as ativações internas do Claude 3 Sonnet e descobiram alguns pontos:
    - O modelo aprende um espaço interno em que as informações similares se agrupam, algo similhante a um embedding interno
    - Esse espaço não é caótico, mas sim organizado em grupos que realmente carregam descrições mesmo que abstratas sobre conceitos reais 
    - Foram encontrados pontos nesse espaço interno que correspondem a conceitos específicos, por exemplo, existe um ponto que representa a golden bridge gate
    - Essas características sao ativadas consistentemente quando o modelo processa informações sobre a ponte, independente do idioma, ou seja, eles aprendem conceitos abstratos que não dependem do idioma que é processado.
  
Pra confirmar a hipótese, eles também fizeram alguns experimentos manipulando esse espaço. Foi identificado um ponto no espaço que corresppndia à caracterísitica neurociência. 
Com o modelo com as ativaçoes padrão, pergumtaram para o Clauda qual era sua ciência preferida, e ele respondeu que era física.
Depois, modificaram o modelo para aumentar a força das ativações encontradas relacionadas a neurociência, e perguntaram a mesma coisa. A resposta agora era que a ciência preferida era neurociência.

Embora ainda não é claro como os modelos aprendem esses padrões, apenas aprendendo a completar tokens, mas essa pesquisa serve como exemplo de como o conhecimento sobre domínios específicos surge nos modelos de linguagem.

In [1]:
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
import matplotlib.pyplot as plt 

if tf.config.list_logical_devices("GPU") != []:
    print("ta c gpu")
else:
    print("ta c cpu")

ta c cpu


In [11]:
def custom_standardization(input_text):
    lowercase_text = tf.strings.lower(input_text)
    cleaned_text = tf.strings.regex_replace(lowercase_text, r"[^a-z0-9 ]", "")
    return cleaned_text

In [ ]:
# Carregando o texto
with open("input.txt", "r", encoding="utf-8") as f:
    content = f.read().lower()

print(content[:100]) #Printando os primeiros 100 caracteres

In [163]:
# Aqui, criamos uma camada de vetorização de texto. Ela tem o papel de converter o texto em um vetor de inteiros. Por exemplo, a palavra "hello" pode ser convertida para [8, 5, 12, 12, 15], caso os índices das letras sejam a sua posição no alfabeto.
text_vec_layer = keras.layers.TextVectorization(
    split="character", #Dividimos o texto em caracteres
    standardize="lower" #Padronizamos o texto para minúsculas
)

#Aqui, adaptamos a camada ao texto. Ela vai criar um índice inteiro para cada caractere usado no texto.
text_vec_layer.adapt([content])
encoded = text_vec_layer([content])[0] # usamos o [0] para pegar o tensor de índices. Utilizamos esse 0 porque a camada de vetorização retorna um tensor de índices, mesmo que só passemos um texto.

In [164]:
# Quando usamos a camada de vetorização, ela adiciona dois tokens especiais: um para o padding e outro para o out-of-vocabulary(caracteres não presentes na tabela de índices). Aqui, subtraímos 2 do número de índices para obter o número de tokens distintos.
encoded -=2
n_tokens = text_vec_layer.vocabulary_size() - 2  
dataset_size = len(encoded)  # Numero de caracteres no texto = 1,782,290

In [165]:
n_tokens, dataset_size

(86, 1782290)

In [166]:
# Hiperparâmetros do dataset
window_size = 128
batch_size = 32

In [ ]:
# Agora, vamos começar a tratar os dados. Lembra que eu falei no início que o que um modelo de linguagem faz é tentar prever o próximo caractere dado um conjunto de caracteres anteriores? Pois bem, vamos criar um dataset que faz exatamente isso.
# Como vimos, o texto é convertido em um vetor de inteiros. Vamos criar um dataset que: a cada passo, pega 128 caracteres e tenta prever o próximo caractere. Ou seja, o input é uma sequência de 128 caracteres e o output é o caractere seguinte.
# Ou seja, o que o modelo vai tentar fazer é prever o próximo índice do vetor 'encoded' dado os 128 índices anteriores.
# Como por exemplo, se a frase for "hello, world", o modelo vai tentar prever o caractere "d" dado "hello, worl".

# Vamos criar um dataset que faz exatamente isso. Para isso, vamos usar a classe tf.data.Dataset. Vamos seguir os seguintes passos:
# Suponha que 'encoded' seja [10, 11, 12, 13, 14] {NUMEROS ARBITRARIOS Q TO USANDO DE EXEMPLO},
# e que window_size=2 (pra simplificar o entendimento)
# No nosso caso real, window_size=128 e a lista 'encoded' é bem maior.

# 1) from_tensor_slices -> transforma 'encoded' em uma sequência de indices na forma de um Dataset (1 caractere/índice por vez)
#    exemplo:
#    encoded = [10, 11, 12, 13, 14]
#    -> Dataset com 5 exemplos: 10, 11, 12, 13, 14
ds = tf.data.Dataset.from_tensor_slices(encoded)

# 2) window -> agrupa a sequência em janelas de tamanho (window_size + 1)
#    Como temos window_size=2 no nosso exemplo, fazemos window_size + 1 = 3
#    Então cada janela terá 3 elementos
#    Shift=1 significa que cada nova janela começa 1 elemento à frente da anterior
#    Ou seja, teremos um dataset de datasets, onde cada dataset interno é uma janela
#    exemplo:
#    [10, 11, 12]
#    [11, 12, 13]
#    [12, 13, 14]
ds = ds.window(window_size + 1, shift=1, drop_remainder=True)

# 3) flat_map -> converte cada janela em um único tensor (em vez de um Dataset de Datasets)
#    Isso vai agrupar cada janela (ex: [10, 11, 12]) em um tensor de shape (3,)
#    exemplo:
#    A janela [10, 11, 12] vira tf.Tensor([10, 11, 12])
#    A janela [11, 12, 13] vira tf.Tensor([11, 12, 13])
ds = ds.flat_map(lambda window: window.batch(window_size + 1))

# 4) shuffle -> embaralha as janelas, para que o modelo não veja tudo em sequência exata
#    exemplo:
#    As janelas [10, 11, 12], [11, 12, 13], [12, 13, 14]
#    podem ser apresentadas em ordem aleatória
ds = ds.shuffle(100_000, seed=42)

# 5) batch -> agrupa várias janelas em um lote (batch).
#    No nosso caso, batch_size=32 no código real, mas no exemplo seria
#    se tivéssemos mais janelas, agruparíamos X janelas em cada batch.
#    exemplo:
#    Se tivéssemos 96 janelas e batch_size=32,
#    teríamos 3 batches, cada um com 32 janelas.
ds = ds.batch(batch_size)

# 6) map -> separa cada janela em (X, y). X é todos os caracteres MENOS o último,
#    y é todos os caracteres MENOS o primeiro. Isso é para termos pares
#    "caractere atual -> próximo caractere"
#    exemplo (continuando):
#    Se a janela for [10, 11, 12], então:
#         X = [10, 11]
#         y = [11, 12]
#    Ou seja, "dado 10, preveja 11; dado 11, preveja 12"
ds = ds.map(lambda window: (window[:, :-1], window[:, 1:]))

# 7) prefetch -> melhorar a perfomance (carrega lotes futuros em paralelo)
#    Não altera logicamente os dados, só otimiza a velocidade
ds = ds.prefetch(1)


In [ ]:
# printando um batch com um exemplo
for X_batch, y_batch in ds.take(1):
    print(X_batch.shape, y_batch.shape)
    print(X_batch[0], y_batch[0])

(32, 128) (32, 128)
tf.Tensor(
[ 0  1 12  1 24  1  0  8  1 10  3  5  1  7  3 16 13  3  0  4  3  8 20  1
  7  3  5  0  7  2  4 22  1 28 29  4  2  0  7  1  0 19 11  6 10  2  5  1
 17 17 17 13  2 14  1  0  3  0  2 33 12  6  9  1 16  0 12  3 10  0 21  2
  4  9  3  0 12  1 14 12 11 14  1  7  3 32 13  8  1  0 12  1 24  2 26  1
  0 14 20  2  0 14  1  8 26  1  0 14  2 18  2 10  2  8  9  2 13  3  0  1
  7  1 10  1  8  9  6  8], shape=(128,), dtype=int64) tf.Tensor(
[ 1 12  1 24  1  0  8  1 10  3  5  1  7  3 16 13  3  0  4  3  8 20  1  7
  3  5  0  7  2  4 22  1 28 29  4  2  0  7  1  0 19 11  6 10  2  5  1 17
 17 17 13  2 14  1  0  3  0  2 33 12  6  9  1 16  0 12  3 10  0 21  2  4
  9  3  0 12  1 14 12 11 14  1  7  3 32 13  8  1  0 12  1 24  2 26  1  0
 14 20  2  0 14  1  8 26  1  0 14  2 18  2 10  2  8  9  2 13  3  0  1  7
  1 10  1  8  9  6  8  3], shape=(128,), dtype=int64)


In [168]:
def to_dataset(sequence, window_size=128, batch_size=32, seed=42, shuffle=False, target='all_window'):
    """ Aqui, modularizando o código acima em uma função """
    ds = tf.data.Dataset.from_tensor_slices(sequence) # Fatia o ds em janelas de tamanho window_size
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True) # Você pode visualizar com um .as_numpy_iterator() e colocar num loop com .take(1)
    ds = ds.flat_map(lambda window: window.batch(window_size + 1)) # Transforma cada janela em um tensor
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    if target == 'all_window':
        ds = ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)
    elif target == 'last_char':
        ds = ds.map(lambda window: (window[:, :-1], window[:, -1])).prefetch(1)
    else:
        raise ValueError('target must be "all_window" or "last_char"')
    return ds

In [170]:
# Vamos criar os datasets de treino, validação e teste
window_size = 128
train_set = to_dataset(encoded[:1_000_000], window_size=window_size, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], window_size=window_size)
test_set = to_dataset(encoded[1_060_000:], window_size=window_size)

In [171]:

f.random.set_seed(42)  # extra code – reproducilidade

#Aqui, nós criamos o modelo: uma camada de embeddings, que vai mapear os índices dos caracteres para vetores de tamanho 16. Depois, temos uma camada de convolução 1D com 32 filtros e kernel_size=3. A seguir, temos uma camada de GRU com 128 unidades. Por fim, temos uma camada densa com ativação softmax, que vai prever o próximo caractere. A saída da última camada é um vetor de tamanho n_tokens, que é o número de caracteres distintos no texto, cada posição do vetor representa a probabilidade do caractere ser o próximo
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.Conv1D(32, kernel_size=3, padding="causal", activation="relu"),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [172]:
def decoder(seq, delta=2):
    """ 
        Aqui, decodificamos a sequência de índices para texto.
        Usa a camada de vetorização para mapear os índices para caracteres, ou seja, o inverso do que a camada de vetorização faz.
        seq: sequência de índices
        delta: deslocamento para mapear os índices para caracteres (default=2, pois tiramos os tokens especiais)
    """
    output = np.array(text_vec_layer.get_vocabulary())[seq + delta]
    new_output = []
    for i in range(output.shape[0]):
        phrase = ""
        for j in range(output.shape[1]):
            phrase += output[i, j]
        new_output.append(phrase)
    return np.array(new_output)

In [173]:
# estilo
plt.style.use('dark_background')
plt.rcParams['axes.facecolor'] = '#26262e' 

def print_multiple_generated(predicted_text, original_text, n=8):
    print("--> Original text:")
    for i in range(n):
        print(original_text[i])
    print("\n--> Generated text:")
    for i in range(n):
        print(predicted_text[i])
        
def plot_loss(history, step):
    plt.plot(list(range(len(history['loss']))), history['loss'], label='Training Loss')
    plt.plot(list(range(len(history['loss']))), history['val_loss'], label='Validation Loss')
    plt.xlabel('Step')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
def plot_accuracy(history, step):
    plt.plot(list(range(len(history['accuracy']))), history['accuracy'], label='Training Accuracy')
    plt.plot(list(range(len(history['accuracy']))), history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Step')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [180]:
# Aqui, criamos um callback personalizado para logar o texto gerado a cada 1000 batches. O callback também loga o final de cada época.

history = {
            'text_base': 'A menina estava ',
            'text_base_encoded': text_vec_layer(['A menina estava ']),
            'list_predicted_text_base': [],
        }

class LogCallback(tf.keras.callbacks.Callback):
    def __init__(self, history):
        super().__init__()
        self.history_ = history
    
    def on_epoch_end(self, epoch, logs=None):
        print(f"Final da época {epoch}, Loss: {logs['loss']}, Acc: {logs['accuracy']}")
    
    def on_train_batch_end(self, batch, logs=None):
        if batch % 1_000 == 0:
            print("")
            print(f"Final do lote {batch}, Loss do lote: {logs['loss']}")
            model = self.model
            predicted_text = extend_text("A menina estava ", temperature=1.0)
            print("")
            print("--> Original text: ", history['text_base'])
            print("--> Generated text: ", predicted_text)
            self.history_['list_predicted_text_base'].append(predicted_text)
            print("")

In [ ]:
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "machado_model/machado_model.keras", monitor="val_accuracy", save_best_only=True)


# compilamos o modelo com a perda de sparse categorical entropy, o otimizador Nadam e a métrica de acurácia. Em seguida, treinamos o modelo com 10 épocas e o callback personalizado.
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="nadam",
              metrics=["accuracy"])

# definimos a sequencial, em que o texto passa pela camada de vetorização, é subtraído 2 e então passa pelo modelo.
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # tira os tokens especiais
    model
])


def extend_text(text, n_chars=50, temperature=1):
    """ 
        Aqui, nós usamos o modelo para gerar texto.
        Dado um texto inicial, o modelo preve o próximo caractere e adiciona ao texto.

        A saida do modelo é um vetor de probabilidades para o próximo caractere, nós normalizamos essas probabilidades com a temperatura.
        Quanto maior a temperatura, mais aleatório o texto gerado será.
        Depois, usamos tf.random.categorical para amostrar o próximo caractere, ou seja, escolher um caractere aleatório baseado nas probabilidades, o que muito provavelmente vai ser o caractere com maior probabilidade.
        Depois, obtemos o caractere correspondente ao índice amostrado e adicionamos ao texto.
        Faremos isso n_chars vezes, gerando um texto de n_chars caracteres.
    """
    for _ in range(n_chars):
        input_data = tf.constant([text], dtype=tf.string)
        y_proba = shakespeare_model.predict(input_data, verbose=0)[0, -1:]
        rescaled_logits = tf.math.log(y_proba) / temperature
        char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0].numpy()
        next_char = text_vec_layer.get_vocabulary()[char_id + 2]
        text += next_char
    return text

In [185]:
# Treinando o modelo
history_train = model.fit(train_set, 
                    validation_data=valid_set, 
                    epochs=10,
                    callbacks=[LogCallback(history=history),
                    model_ckpt])

Epoch 1/10

Final do lote 0, Loss do lote: 1.8213555812835693

--> Original text:  A menina estava 
--> Generated text:  A menina estava se
felita na págio
terrível artustes como
quãe era

   1000/Unknown 74s 47ms/step - accuracy: 0.5161 - loss: 1.5810
Final do lote 1000, Loss do lote: 1.5270321369171143

--> Original text:  A menina estava 
--> Generated text:  A menina estava minh’alma,
jou-lhes que minha mãe que se te proscr

   2000/Unknown 121s 47ms/step - accuracy: 0.5262 - loss: 1.5456
Final do lote 2000, Loss do lote: 1.499700903892517

--> Original text:  A menina estava 
--> Generated text:  A menina estava da man! — um digado
sardima, e propa em que pariet

   2999/Unknown 168s 47ms/step - accuracy: 0.5311 - loss: 1.5283
Final do lote 3000, Loss do lote: 1.4871782064437866

--> Original text:  A menina estava 
--> Generated text:  A menina estava que encontrar, político?
olhará se flores. hausem 

   4000/Unknown 217s 47ms/step - accuracy: 0.5344 - loss: 1.5168
Final do lote

c:\Users\gugu1\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


Final da época 0, Loss: 1.4377723932266235, Acc: 0.5614280104637146
31246/31246 ━━━━━━━━━━━━━━━━━━━━ 2305s 73ms/step - accuracy: 0.5549 - loss: 1.4563 - val_accuracy: 0.5110 - val_loss: 1.6662
Epoch 2/10

Final do lote 0, Loss do lote: 2.0438084602355957

--> Original text:  A menina estava 
--> Generated text:  A menina estava e dramática. iludivedo esposar na transição em, di

  999/31246 ━━━━━━━━━━━━━━━━━━━━ 24:05 48ms/step - accuracy: 0.4883 - loss: 1.6952
Final do lote 1000, Loss do lote: 1.5996482372283936

--> Original text:  A menina estava 
--> Generated text:  A menina estava as vitórias poedas
diante;
nuar como abagar, e nas

 1999/31246 ━━━━━━━━━━━━━━━━━━━━ 23:38 48ms/step - accuracy: 0.5032 - loss: 1.6328
Final do lote 2000, Loss do lote: 1.5501521825790405

--> Original text:  A menina estava 
--> Generated text:  A menina estava alma gienlão vento de beijador socia,
e que un rec

 3000/31246 ━━━━━━━━━━━━━━━━━━━━ 23:01 49ms/step - accuracy: 0.5109 - loss: 1.6008
Final do 